## Notebook to plot ACC Seizure Detection Plots

This notebook is focused solely on FBTC seizures.

A patient-specific approach will be tested

SET 2
- Training: BLIW_1, BLIW_2, YIVL_0, YIVL_1, AGGA_0, AGGA_1, AGGA_2, YWJN_5, YWJN_6 
- Testing: VNVW_1, WOSQ_2, AGGA_3, YIVL_2,  BLIW_3, YWJN_7

In [11]:
# import os
from datetime import datetime

# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.feature_selection import SelectPercentile, f_classif

from fast_ml.feature_selection import get_constant_features

from preepiseizures.src import Patient

## 1. Open data

In [12]:
data = pd.read_parquet('./data/features/all_patients_acc_features.parquet')

print(data.shape)

# remove columns with nans
data = data.dropna(axis=1)
print(data.shape)
data['patient'] = data['patient_seizure'].apply(lambda x: x.split('_')[0])
data['patient'].unique()

training_set_keys = ['BLIW_0', 'BLIW_1', 'BLIW_2', 'YIVL_0', 'YIVL_1', 'AGGA_0', 'AGGA_1', 'AGGA_2', 'YWJN_4', 'YWJN_5', 'YWJN_6'] 
testing_set_keys = ['VNVW_0', 'VNVW_1', 'WOSQ_2', 'AGGA_3', 'YIVL_2',  'BLIW_3', 'YWJN_7']
all_training_set = data[data['patient_seizure'].isin(training_set_keys)].copy()
all_testing_set = data[data['patient_seizure'].isin(testing_set_keys)].copy()

features_cols = all_training_set.drop(columns=['patient_seizure', 'patient', 'timestamp', 'y']).columns

(87810, 1559)
(87810, 1557)


### 2.2 Select patient data

In [13]:
patient = 'AGGA'
training_set = all_training_set[all_training_set['patient'] == patient].copy()
testing_set = all_testing_set[all_testing_set['patient'] == patient].copy()

train_seizures = training_set['patient_seizure'].unique()
test_seizures = testing_set['patient_seizure'].unique()
print(training_set.shape, testing_set.shape)

patient_ = Patient.patient_class(patient)
patient_.get_seizure_annotations()

seiz_surrounding_index = []

for seizure in patient_.seizure_table.index:

    onset = patient_.seizure_table.loc[seizure]['Timestamp']
    # preictal period is 10 minutes before seizure
    start = onset - pd.Timedelta(minutes=10)
    try:
        end_time = datetime.combine(onset.date(), patient_.seizure_table.loc[seizure, 'Fim'])
    except:
        end_time = onset + pd.Timedelta(minutes=2)
    seiz_surrounding_index += [training_set.loc[training_set['timestamp'].between(start, onset - pd.Timedelta(seconds=30))].index]
    posictal_0 = end_time
    posictal_1 = onset + pd.Timedelta(minutes=10)
    seiz_surrounding_index += [training_set.loc[training_set['timestamp'].between(posictal_0, posictal_1)].index]
    
seiz_surrounding_index = np.hstack(seiz_surrounding_index)      

training_set.drop(seiz_surrounding_index, inplace=True)

constant_features = get_constant_features(training_set[features_cols])
# print(constant_features)
exclude_cols = constant_features.Var
training_set.drop(columns=exclude_cols, inplace=True)
print('Number of features excluded: ', len(exclude_cols))
features_cols = training_set.drop(columns=['patient_seizure', 'patient', 'timestamp', 'y']).columns

corr_matrix = training_set[features_cols].corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(f'Dropping {len(to_drop)} features from {corr_matrix.shape[1]}.')

# Drop features 
#if to_drop[0] in training_set.columns:
#    training_set.drop(to_drop, axis=1, inplace=True)
new_training_set = training_set.drop(columns=to_drop)
new_testing_set = testing_set.drop(columns=to_drop)
features_cols = new_training_set.drop(columns=['patient_seizure', 'timestamp', 'y', 'patient']).columns

print('New shape: ', new_training_set.shape)
print(len(features_cols))

scaler = MinMaxScaler()
scaler.fit(new_training_set[features_cols])
new_training_set[features_cols] = scaler.transform(new_training_set[features_cols])
new_testing_set[features_cols] = scaler.transform(new_testing_set[features_cols])


(3264, 1558) (2013, 1558)
Number of features excluded:  58
Dropping 1214 features from 1496.
New shape:  (2821, 286)
282


## Features Selection

In [14]:

X_train_ = new_training_set[features_cols].copy()
y_train_ = new_training_set['y'].copy()
X_test_ = new_testing_set[features_cols].copy()
y_test_ = new_testing_set['y'].copy()


percentile = 2
selector_anova = SelectPercentile(score_func=f_classif, percentile=percentile)
X_train_ANOVA = selector_anova.fit_transform(X_train_, y_train_)
X_test_ANOVA = selector_anova.transform(X_test_)


feature_index = selector_anova.get_support(indices=True)
feature_names = X_train_.columns[feature_index]

# Convert selected features to DataFrame
features_anova_train1 = pd.DataFrame(X_train_ANOVA, columns=feature_names, index=X_train_.index)
features_anova_test1 = pd.DataFrame(X_test_ANOVA, columns=feature_names, index=X_test_.index)

print('Number of features selected: ', len(feature_names))

training_set_keys = new_training_set['patient_seizure'].unique()

validation_set_keys = [training_set_keys[-1]]
training_set_keys_subset = training_set_keys[:-1]

X_train = features_anova_train1.loc[new_training_set.loc[new_training_set['patient_seizure'].isin(training_set_keys_subset)].index]
y_train = y_train_.loc[new_training_set.loc[new_training_set['patient_seizure'].isin(training_set_keys_subset)].index]
X_val = features_anova_train1.loc[new_training_set.loc[new_training_set['patient_seizure'].isin(validation_set_keys)].index]
y_val = y_train_.loc[new_training_set.loc[new_training_set['patient_seizure'].isin(validation_set_keys)].index]

# X_train, X_test, y_train, y_test = train_test_split(X_train_ANOVA, y_train_, test_size=0.2, random_state=42)

# Handle class imbalance using SMOTE for oversampling and NearMiss for undersampling
oversampler = SMOTE(sampling_strategy=0.5)  # Adjust the sampling strategy as needed
undersampler = NearMiss()
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)
print(X_train_resampled.shape, y_train_resampled.shape)
# print number of samples in each class
print('Class 0: ', len(y_train_resampled[y_train_resampled == 0]))
print('Class 1: ', len(y_train_resampled[y_train_resampled == 1]))


# Define a list of classifiers to test
classifiers = [
    ('Random Forest', RandomForestClassifier()),
    ('SVM', SVC()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Logistic Regression', LogisticRegression())
]

Number of features selected:  6
(2086, 6) (2086,)
Class 0:  1043
Class 1:  1043


In [15]:
def decision_layer(df, life):
    """
    Decision layer to detect a seizure
    :param df: dataframe with y_pred column, timestamp column and index
    :return: index of seizure
    """
    consecutive_preds = 0
    seizure_counter_dict = {}
    # all index of N consecutive 1s
    # add a median filter to smooth the prediction
    i = 0
    alarm = False
    
    while i < len(df):
        # in an alarm is raised
        if (consecutive_preds >= life and not alarm):
            # raise an alarm
            seizure_counter_dict[df['time'].iloc[i]] = i
            alarm = True
            # wait for 
            # print('Seizure detected at index: ', i)
        else:
            if df['y_pred'].iloc[i] == 1:
                consecutive_preds += 1
            if df['y_pred'].iloc[i] == 0:
                consecutive_preds = 0
                alarm = False
        i += 1
    # after a consec_index wait for 2 minutes before the next prediction
    return seizure_counter_dict

def eval_alarms(preds, seizure):
    """
    Evaluate the alarms
    :param preds: list of predictions
    :param seizure: seizure dataframe
    """
    true_alarm, false_alarm = 0, 0
    if type(preds) == dict:
        preds = list(preds.keys())
    
    if seizure is None:
        # every alarm is a false alarm
        return 0, len(preds)
    
    seizure_range_start = seizure['Timestamp'] - pd.Timedelta(seconds=30)
    seizure_range_end = seizure['Timestamp'] + pd.Timedelta(seconds=180)


    for alarm in preds:
        if (seizure_range_start <= alarm) and (alarm <= seizure_range_end):
            print('Seizure detected at: ', alarm - seizure['Timestamp'])
            true_alarm += 1
        else:
            false_alarm += 1
    
    return true_alarm, false_alarm


def eval_models(classifiers, X_train_resampled, y_train_resampled, X_val, y_val, new_training_set, new_testing_set, patient_):
    """
    Evaluate the model
    :return: None
    """
    
    # Train and evaluate each classifier
    for name, classifier in classifiers:
        classifier.fit(X_train_resampled, y_train_resampled)
        y_pred = classifier.predict(X_val)
        recall = recall_score(y_val, y_pred)
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        # Print classification report and other metrics
        print(f"Classifier: {name} | Recall: {np.round(100*recall, 2)} | Accuracy: {np.round(100*accuracy, 2)} | Precision: {np.round(100*precision, 2)} | F1-score: {np.round(100*f1, 2)}")

        #EVENT BASED ANALYSIS
        y_pred = classifier.predict(X_val)
        y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'], index=y_val.index)
        y_pred_df['time'] = new_training_set.loc[y_pred_df.index, 'timestamp']
        y_pred_df.sort_index(inplace=True)
        # median filter to smooth the predictions

        if precision < 0.1:
            median = 10
        else:
            median = 10
        y_pred_df['y_pred'] = y_pred_df['y_pred'].rolling(median, center=True).median()

        val_alarms = decision_layer(y_pred_df, life=10)

        val_seizure_idx = int(validation_set_keys[0].split('_')[-1])
        val_seizure = patient_.seizure_table.loc[val_seizure_idx]
        tp, fp = eval_alarms(val_alarms, val_seizure)
        val_time = (y_pred_df['time'].iloc[-1] - y_pred_df['time'].iloc[0]).total_seconds()/3600
        fp_rate = np.round(fp/val_time,2)
        print(f'VAL SD: {tp}/1 | VAL FAR: {fp_rate}/h')

        #TESTING SET
        y_pred_test = classifier.predict(features_anova_test1)

        y_pred_df_test = pd.DataFrame(y_pred_test, columns=['y_pred'], index=new_testing_set.index)
        y_pred_df_test['time'] = new_testing_set['timestamp']

        y_pred_df_test['y_pred'] = y_pred_df_test['y_pred'].rolling(median, center=True).median()


        test_alarms = decision_layer(y_pred_df_test, life=10)
        
        test_seizures = new_testing_set['patient_seizure'].unique()
        if len(test_seizures) > 1:
            print('deal with multiple seizures')
        else:
            test_seizure_idx = int(test_seizures[0].split('_')[-1])
        test_seizure = patient_.seizure_table.loc[test_seizure_idx]
        if new_testing_set['y'].sum() == 0:
            print('No seizure in testing set')
            test_seizure = None
        if y_val.sum() == 0:
            print('No seizure in validation set')

        tp, fp = eval_alarms(test_alarms, test_seizure)
        
        test_time = (y_pred_df_test['time'].iloc[-1] - y_pred_df_test['time'].iloc[0]).total_seconds()/3600
        
        fp_rate = np.round(fp/test_time,2)


        print(f'TEST SD: {tp}/{len(test_seizures)} | TEST FAR: {fp_rate}/h')
        print('\n')
        


In [16]:
eval_models(classifiers, X_train_resampled, y_train_resampled, X_val, y_val, new_training_set, new_testing_set, patient_)

Classifier: Random Forest | Recall: 2.5 | Accuracy: 93.17 | Precision: 9.09 | F1-score: 3.92
VAL SD: 0/1 | VAL FAR: 0.0/h
TEST SD: 0/1 | TEST FAR: 0.0/h


Classifier: SVM | Recall: 10.0 | Accuracy: 80.47 | Precision: 3.7 | F1-score: 5.41
VAL SD: 0/1 | VAL FAR: 0.0/h
TEST SD: 0/1 | TEST FAR: 0.34/h


Classifier: Decision Tree | Recall: 2.5 | Accuracy: 90.1 | Precision: 3.03 | F1-score: 2.74
VAL SD: 0/1 | VAL FAR: 0.0/h
TEST SD: 0/1 | TEST FAR: 0.67/h


Classifier: Logistic Regression | Recall: 60.0 | Accuracy: 66.25 | Precision: 9.6 | F1-score: 16.55
VAL SD: 0/1 | VAL FAR: 3.23/h
TEST SD: 0/1 | TEST FAR: 2.02/h




In [18]:
new_testing_set['patient_seizure'].unique()

array(['AGGA_3'], dtype=object)